Install libraries required to scrape the webpage

In [153]:
from bs4 import BeautifulSoup
import pandas as pd

Read the webpage from URL

In [154]:
import requests

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
pageSource = requests.get(url)

Create lists to capture required elements from the webpage

In [155]:
tableData=[] 
PostalCode=[] 
Borough=[]
Neighborhood=[]

Retrieve the webpage HTML content

In [156]:
soup = BeautifulSoup(pageSource.content)

Parse the required data from HTML and assign to respective Lists

In [157]:
indx = 1
for tbody in soup.findAll('td', attrs={'style':''}):
    if (indx / 1) == 1:
        PostalCode.append(tbody.text.replace("\n",""))
    elif (indx / 2) == 1:
        Borough.append(tbody.text.replace("\n",""))
    elif (indx / 3) == 1:
        Neighborhood.append(tbody.text.replace("\n",""))
    
    if indx > 2:
        indx = 1
    else:
        indx = indx + 1
    
#print(PostalCode)
#print(Borough)
#print(Neighborhood)

Fulfilling following condition: The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood.

In [158]:
# Assign the Lists data to DataFrame
df = pd.DataFrame({'PostalCode':PostalCode,'Borough':Borough,'Neighborhood':Neighborhood})
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Fufilling following condition: Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned. 

In [169]:
# Exlude cells that has Borough as 'Not assigned'
df = df[df['Borough'] != "Not assigned"]
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Fufilling following condition: More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [160]:
# Group by PostCode and combine multiple neighborhood into single row
df["Neighborhood"] = df.groupby(["PostalCode","Borough"], sort=False)["Neighborhood"].transform(
    lambda x: "," .join(x)
)
df = df.drop_duplicates()
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


There are no Borough that has Neighborhood as 'Not assigned'. Hence, no such clean up required

Print number of rows using .shape method

In [161]:
print("Number of rows in the dataframe: ", df.shape[0])

Number of rows in the dataframe:  103


In [162]:
# Download the CSV file having coordinates data
df_GeoData = pd.read_csv('http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv')
df_GeoData.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Create a new dataframe as specified in Assignment

In [164]:
# Create a consolidated dataframe

# Make the column name same for existing and GeoData dataframe
df_GeoData = df_GeoData.rename(columns={"Postal Code":"PostalCode"})

# Merge both the data frame to include Latitude and Longitude
df_merged = pd.merge(df, df_GeoData, on="PostalCode")
df_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
